# Creating the WebScraper using Selenium to gather Geeks Who Drink(GWD) data in Texas

In [1]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import time as time

import selenium
from selenium import webdriver

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

In [2]:
path_to_driver = "../chromedriver_win32/chromedriver.exe"

In [3]:
#calling the webdriver for viz
browser = webdriver.Chrome(executable_path=path_to_driver)

In [4]:
url = 'https://www.geekswhodrink.com/schedule/TX'

In [5]:
#finding the URL
browser.get(url)

In [6]:
#Use this to wait for the site to load.
time.sleep(8)

In [7]:
#setting the data to be collected from the sites.
data ={}
dow = []
venue = []
city = []
time = []
urls = []
date = []

In [8]:
days = [day.text for day in browser.find_elements_by_tag_name("h3")[1:]]

In [9]:
#Using 0 as a beginning date because div 0 doesn't exist. Use it as placeholder to interate through the next DOW
for l in range(len(days)):
    print(l,days[l])

0 
1 Sunday
2 Monday
3 Tuesday
4 Wednesday
5 Thursday
6 Saturday


## Scraping the Site

In [10]:
#finds the urls between each h3 header
##tqdm is used for the statusbar
for stuff in tqdm(range(len(days))):
    for i in browser.find_elements_by_xpath('/html/body/div/div/div/div[2]/div[1]/div/div/div/div['+str(stuff)+']'):
        for links in i.find_elements_by_tag_name("a"):
            dow.append(days[stuff])
            urls.append(links.get_attribute("href"))
            venue.append(links.text)
        for c in i.find_elements_by_tag_name("i"):
            city.append(c.text)
        for t in i.find_elements_by_tag_name("time"):
            if (t.text[0]).isnumeric() == True:
                time.append(t.text)
            else: 
                date.append(t.text)

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:17<00:00,  2.49s/it]


In [11]:
#checks to ensure that data is the same length
len(city),len(venue),len(dow),len(time),len(urls)

(177, 177, 177, 177, 177)

### Constructing the DataFrame

In [12]:
df = pd.DataFrame(data=[city,venue,dow,time,urls]).T

In [13]:
df.columns = ['city','venue','dow','time','urls']
browser.close()

In [14]:
df['dow'].value_counts()

Tuesday      54
Wednesday    50
Thursday     32
Monday       26
Sunday        9
Saturday      6
Name: dow, dtype: int64

### Checking all of the Austin locations

In [15]:
austin  = df[df['city'] == '(Austin)']

In [16]:
#this is the data as a whole
## use a datestamp to ensure that there isn't data overwritten unless it's on the same day.
df.to_csv('./data/'+str(datetime.today())[:10]+'_GWD.csv')

In [17]:
#using this to match the indexes for Tableau
austin = austin.reset_index(drop=True)

In [18]:
austin.to_csv('./data/'+str(datetime.today())[:10]+'_GWD_Austin.csv')